<h1> 1. Download yolov3-tiny.weights, yolov3-tiny.cfg file</h1>

[download link](https://github.com/smarthomefans/darknet-test/blob/master/yolov3-tiny.weights)

<h1>2. Edit yolov3-tiny.cfg file</h1>

* make [net] as comment
<p> # [net]
<p> # # Testing
<p> # batch=1
<p> # subdivisions=1
<p> # # Training
<p> # # batch=64
<p> # # subdivisions=2
<p> # width=416
<p> # height=416
<p> # channels=3
<p> # momentum=0.9
<p> # decay=0.0005
<p> # angle=0
<p> # saturation = 1.5
<p> # exposure = 1.5
<p> # hue=.1
<p> 
<p> # learning_rate=0.001
<p> # burn_in=1000
<p> # max_batches = 500200
<p> # policy=steps
<p> # steps=400000,450000
<p> # scales=.1,.1

<h1> Read file in JSON

In [2]:
import numpy as np
import json
import os

# 1. 가중치 파일 읽기
def load_weights(file_path):
    with open(file_path, "rb") as f:
        header = np.fromfile(f, dtype=np.int32, count=5)  # 헤더 읽기 (5개 정수)
        weights = np.fromfile(f, dtype=np.float32)        # 나머지 가중치 읽기
    return header, weights

# 2. YOLO 구성 파일 파싱
def parse_cfg(cfg_file):
    layers = []
    with open(cfg_file, "r") as f:
        lines = f.read().splitlines()
        lines = [x.strip() for x in lines if x and not x.startswith("#")]  # 주석 제거
        layer = {}
        for line in lines:
            if line.startswith("["):  # 새 레이어 정의 시작
                if layer:
                    layers.append(layer)  # 이전 레이어 저장
                layer = {"type": line[1:-1]}  # 레이어 타입
            else:
                key, value = line.split("=")
                layer[key.strip()] = value.strip()
        layers.append(layer)  # 마지막 레이어 추가
    return layers

# 3. 레이어별로 가중치를 매핑
def load_weights_to_layers(layers, weights):
    ptr = 0
    layer_weights = []
    previous_output_channels = 3  # 초기 입력 채널 (RGB 이미지)

    for layer_idx, layer in enumerate(layers):
        if layer["type"] == "convolutional":  # Conv 레이어 처리
            filters = int(layer["filters"])
            size = int(layer["size"])
            stride = int(layer.get("stride", 1))
            pad = int(layer.get("pad", 0))
            activation = layer.get("activation", "linear")

            # 입력 채널 동적 계산
            input_channels = previous_output_channels

            # Conv2D 가중치 추출
            weight_size = filters * input_channels * size * size
            conv_weights = weights[ptr:ptr + weight_size].reshape((filters, input_channels, size, size))
            ptr += weight_size

            # 커널 연결 관계 주석 생성
            kernel_connections = [
                f"Output channel {f_idx} is connected to input channel {c_idx}"
                for f_idx in range(filters)
                for c_idx in range(input_channels)
            ]

            if "batch_normalize" in layer:
                # BatchNorm 파라미터 (scale, bias, mean, variance)
                bn_weights = weights[ptr:ptr + 4 * filters].reshape((4, filters))
                ptr += 4 * filters
                layer_weights.append({
                    "type": "convolutional",
                    "layer_index": layer_idx,
                    "filters": filters,
                    "input_channels": input_channels,
                    "kernel_size": size,
                    "stride": stride,
                    "padding": pad,
                    "activation": activation,
                    "conv_weights": conv_weights.tolist(),
                    "batch_normalize": True,
                    "bn_weights": {
                        "scale": bn_weights[0].tolist(),
                        "bias": bn_weights[1].tolist(),
                        "mean": bn_weights[2].tolist(),
                        "variance": bn_weights[3].tolist()
                    },
                    "kernel_connections": kernel_connections  # 연결 관계 주석
                })
            else:
                # Bias 추출
                bias_weights = weights[ptr:ptr + filters]
                ptr += filters
                layer_weights.append({
                    "type": "convolutional",
                    "layer_index": layer_idx,
                    "filters": filters,
                    "input_channels": input_channels,
                    "kernel_size": size,
                    "stride": stride,
                    "padding": pad,
                    "activation": activation,
                    "conv_weights": conv_weights.tolist(),
                    "batch_normalize": False,
                    "bias_weights": bias_weights.tolist(),
                    "kernel_connections": kernel_connections  # 연결 관계 주석
                })

            # 다음 레이어의 입력 채널 업데이트
            previous_output_channels = filters

        else:
            # Conv 레이어가 아닌 경우
            layer_weights.append({"type": layer["type"], "comment": "No weights for this layer."})

    return layer_weights

# 4. JSON으로 저장
def save_weights_to_json(layer_weights, output_file):
    with open(output_file, "w") as f:
        json.dump({"layers": layer_weights}, f, indent=4)

# 5. 경로 설정 및 실행
cfg_file = "yolov3-tiny.cfg"             # YOLOv3-tiny 구성 파일 경로
weights_file = "yolov3-tiny.weights"     # YOLOv3-tiny 가중치 파일 경로
json_output_file = "yolov3-tiny_weights_with_connections.json"  # JSON 출력 파일

# 실행
header, weights = load_weights(weights_file)  # 가중치 읽기
layers = parse_cfg(cfg_file)                  # 구성 파일 파싱
layer_weights = load_weights_to_layers(layers, weights)  # 가중치 매핑

# JSON 저장
save_weights_to_json(layer_weights, json_output_file)

print(f"Weights saved to JSON with connections: {json_output_file}")


Weights saved to JSON with connections: yolov3-tiny_weights_with_connections.json


<h1> Read file in Numpy

In [3]:
import numpy as np
import os

# 1. 가중치 파일 읽기
def load_weights(file_path):
    with open(file_path, "rb") as f:
        header = np.fromfile(f, dtype=np.int32, count=5)  # 헤더 읽기 (5개 정수)
        weights = np.fromfile(f, dtype=np.float32)        # 나머지 가중치 읽기
    return header, weights

# 2. YOLO 구성 파일 파싱
def parse_cfg(cfg_file):
    layers = []
    with open(cfg_file, "r") as f:
        lines = f.read().splitlines()
        lines = [x.strip() for x in lines if x and not x.startswith("#")]  # 주석 제거
        layer = {}
        for line in lines:
            if line.startswith("["):  # 새 레이어 정의 시작
                if layer:
                    layers.append(layer)  # 이전 레이어 저장
                layer = {"type": line[1:-1]}  # 레이어 타입
            else:
                key, value = line.split("=")
                layer[key.strip()] = value.strip()
        layers.append(layer)  # 마지막 레이어 추가
    return layers

# 3. 레이어별로 가중치를 매핑
def load_weights_to_layers(layers, weights):
    ptr = 0
    layer_weights = []
    previous_output_channels = 3  # 초기 입력 채널 (RGB 이미지)

    for layer in layers:
        if layer["type"] == "convolutional":  # Conv 레이어 처리
            filters = int(layer["filters"])
            size = int(layer["size"])
            stride = int(layer.get("stride", 1))
            pad = int(layer.get("pad", 0))
            activation = layer.get("activation", "linear")

            # 입력 채널 동적 계산
            input_channels = previous_output_channels

            # Conv2D 가중치 추출
            weight_size = filters * input_channels * size * size
            conv_weights = weights[ptr:ptr + weight_size].reshape((filters, input_channels, size, size))
            ptr += weight_size

            if "batch_normalize" in layer:
                # BatchNorm 파라미터 (scale, bias, mean, variance)
                bn_weights = weights[ptr:ptr + 4 * filters].reshape((4, filters))
                ptr += 4 * filters
                layer_weights.append({
                    "conv_weights": conv_weights,
                    "bn_weights": {
                        "scale": bn_weights[0],
                        "bias": bn_weights[1],
                        "mean": bn_weights[2],
                        "variance": bn_weights[3]
                    },
                    "activation": activation,
                    "stride": stride,
                    "padding": pad
                })
            else:
                # Bias 추출
                bias_weights = weights[ptr:ptr + filters]
                ptr += filters
                layer_weights.append({
                    "conv_weights": conv_weights,
                    "bias_weights": bias_weights,
                    "activation": activation,
                    "stride": stride,
                    "padding": pad
                })

            # 다음 레이어의 입력 채널 업데이트
            previous_output_channels = filters

        else:
            # Conv 레이어가 아닌 경우
            layer_weights.append(None)

    return layer_weights

# 4. 가중치를 NumPy 파일로 저장
def save_weights_to_numpy(layer_weights, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i, weights in enumerate(layer_weights):
        if weights is not None:
            np.save(os.path.join(output_dir, f"layer_{i}_conv.npy"), weights["conv_weights"])
            if "bn_weights" in weights:
                for key, value in weights["bn_weights"].items():
                    np.save(os.path.join(output_dir, f"layer_{i}_bn_{key}.npy"), value)
            if "bias_weights" in weights:
                np.save(os.path.join(output_dir, f"layer_{i}_bias.npy"), weights["bias_weights"])

# 5. 경로 설정 및 실행
cfg_file = "yolov3-tiny.cfg"             # YOLOv3-tiny 구성 파일 경로
weights_file = "yolov3-tiny.weights"     # YOLOv3-tiny 가중치 파일 경로
numpy_output_dir = "yolov3-tiny_numpy_weights"  # NumPy 출력 디렉토리

# 실행
header, weights = load_weights(weights_file)  # 가중치 읽기
layers = parse_cfg(cfg_file)                  # 구성 파일 파싱
layer_weights = load_weights_to_layers(layers, weights)  # 가중치 매핑

# 결과 저장 (NumPy)
save_weights_to_numpy(layer_weights, numpy_output_dir)

print(f"Weights saved to NumPy files: {numpy_output_dir}")


Weights saved to NumPy files: yolov3-tiny_numpy_weights


<h1> Extract Numpy to txt(16b fixed point)

In [5]:
import os
import numpy as np

layer = 0  # 처리할 레이어 인덱스

# 고정소수점 변환에 사용될 스케일링 값
FIXED_POINT_SCALE = 2**8  # 고정소수점 스케일링 (소수점 이하 8비트)

def to_fixed_point(value, scale):
    """
    실수를 고정소수점 16진수로 변환
    - value: 변환할 실수 값 배열
    - scale: 고정소수점 스케일링 값 (예: 2^8)
    """
    scaled = np.round(value * scale).astype(int)  # 스케일링 후 정수 변환
    hex_value = np.where(
        scaled >= 0,
        scaled,
        (1 << 16) + scaled,  # 음수는 16비트 2의 보수 처리
    )
    return [f"{x:04X}" for x in hex_value]  # 4자리 16진수 문자열 반환

def generate_kernel_connections(out_channels, in_channels):
    """
    각 출력 채널과 입력 채널 간의 연결 정보를 생성
    - out_channels: 출력 채널 개수
    - in_channels: 입력 채널 개수
    """
    connections = []
    for out_ch in range(out_channels):
        for in_ch in range(in_channels):
            connections.append(f"Output channel {out_ch} is connected to input channel {in_ch}")
    return connections


# 1. Conv 레이어 가중치 및 편향 로드
try:
    if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_conv.npy"):
        conv_weights = np.load(f"yolov3-tiny_numpy_weights/layer_{layer}_conv.npy")
        print("Conv Weights Shape:", conv_weights.shape)

        out_channels, in_channels, kernel_h, kernel_w = conv_weights.shape
        kernel_size = kernel_h * kernel_w  # 한 커널당 요소 개수
        kernel_connections = generate_kernel_connections(out_channels, in_channels)

        conv_weights_flat = conv_weights.flatten()
        conv_weights_hex = to_fixed_point(conv_weights_flat, FIXED_POINT_SCALE)

        with open(f"layer_{layer}_conv_weights_fixed.txt", "w") as f:
            kernel_index = 0
            for i in range(0, len(conv_weights_flat), kernel_size):
                kernel_values = conv_weights_flat[i:i + kernel_size]
                kernel_hex = conv_weights_hex[i:i + kernel_size]

                for original, hex_value in zip(kernel_values, kernel_hex):
                    f.write(f"{hex_value} // {original:.6f}\n")

                connection_comment = kernel_connections[kernel_index]
                f.write(f"// {connection_comment}\n\n")
                kernel_index += 1

    if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bias.npy"):
        conv_bias = np.load(f"yolov3-tiny_numpy_weights/layer_{layer}_bias.npy")
        conv_bias_hex = to_fixed_point(conv_bias, FIXED_POINT_SCALE)
        with open(f"layer_{layer}_conv_bias_fixed.txt", "w") as f:
            for original, hex_value in zip(conv_bias, conv_bias_hex):
                f.write(f"{hex_value} // {original:.6f}\n")
except Exception as e:
    print(f"Error processing Conv weights or biases: {e}")


# 2. BatchNorm 파라미터 로드
try:
    if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_scale.npy"):
        scale = np.load(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_scale.npy")
        scale_hex = to_fixed_point(scale, FIXED_POINT_SCALE)
        with open(f"layer_{layer}_bn_scale_fixed.txt", "w") as f:
            for original, hex_value in zip(scale, scale_hex):
                f.write(f"{hex_value} // {original:.6f}\n")

    if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_bias.npy"):
        bias = np.load(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_bias.npy")
        bias_hex = to_fixed_point(bias, FIXED_POINT_SCALE)
        with open(f"layer_{layer}_bn_bias_fixed.txt", "w") as f:
            for original, hex_value in zip(bias, bias_hex):
                f.write(f"{hex_value} // {original:.6f}\n")

    if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_mean.npy"):
        mean = np.load(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_mean.npy")
        mean_hex = to_fixed_point(mean, FIXED_POINT_SCALE)
        with open(f"layer_{layer}_bn_mean_fixed.txt", "w") as f:
            for original, hex_value in zip(mean, mean_hex):
                f.write(f"{hex_value} // {original:.6f}\n")

    if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_variance.npy"):
        variance = np.load(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_variance.npy")
        variance_hex = to_fixed_point(variance, FIXED_POINT_SCALE)
        with open(f"layer_{layer}_bn_variance_fixed.txt", "w") as f:
            for original, hex_value in zip(variance, variance_hex):
                f.write(f"{hex_value} // {original:.6f}\n")
except Exception as e:
    print(f"Error processing BatchNorm parameters: {e}")


Conv Weights Shape: (16, 3, 3, 3)


<h1> Make layer # to C header

In [18]:
# before, should run numpy to txt code

layer = 0
# conv 생성 부분 따로 편집하기
try: 
    with open(f"../C/header/layer_{layer}_conv_param.h", "w") as f:
        if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_conv.npy"):
            f.write(f"// Conv layer {layer} weights\n")
            f.write(f"s16 layer{layer}_conv_weight[][] = {{{{\n")
            for i in range(0, len(conv_weights_flat), kernel_size):
                temp = 0
                kernel_values = conv_weights_flat[i:i + kernel_size]
                kernel_hex = conv_weights_hex[i:i + kernel_size]
                for hex_value in kernel_hex:
                    temp += 1 
                    if(temp == len(kernel_hex)):
                        f.write(f"{hex_value}\n")
                    else: 
                        f.write(f"{hex_value},\n")

                if i + kernel_size >= len(conv_weights_flat):  
                    f.write("}\n")           
                else: 
                    f.write("},\n") 
            f.write("};\n") 

except Exception as e:
    print(f"Error processing Conv weights or biases: {e}")


try:
    with open(f"../C/header/layer_{layer}_bn_param.h", "w") as f:
        if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_scale.npy"):
            f.write("// bn scale\n")
            f.write(f"s16 layer{layer}_bn_scale[] = {{\n")
            for hex_val in scale_hex:  
                f.write(f"{hex_val},\n")  
            f.write("};\n")  

        if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_bias.npy"):
            f.write("// bn bias\n")
            f.write(f"s16 layer{layer}_bn_bias[] = {{\n")
            for hex_val in bias_hex:  
                f.write(f"{hex_val},\n")  
            f.write("};\n") 

        if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_mean.npy"):
            f.write("// bn mean\n")
            f.write(f"s16 layer{layer}_bn_mean[] = {{\n")
            for hex_val in mean_hex:  
                f.write(f"{hex_val},\n")  
            f.write("};\n") 

        if os.path.exists(f"yolov3-tiny_numpy_weights/layer_{layer}_bn_variance.npy"):
            f.write("// bn variance\n")
            f.write(f"s16 layer{layer}_bn_variance[] = {{\n")
            for hex_val in variance_hex:  
                f.write(f"{hex_val},\n")  
            f.write("};\n") 
        


except Exception as e:
    print(f"Error processing BatchNorm parameters: {e}")
